In [1]:
cd ../..

/home/msi/projects/diplomka


In [2]:
from trct.models import NormalModel

import pandas as pd
import pathlib
from datetime import datetime
import glob
import matplotlib.pyplot as plt

In [3]:
model_name = 'NormalModel'

In [4]:
dataset = glob.glob("results/benchmark/RTT/*.csv", recursive=True)
benchmark_dir = pathlib.Path('results/benchmark/')

In [5]:
def load_df(path):
    df = pd.read_csv(path)
    df['timestamp'] = pd.to_datetime(df['timestamp'] if 'timestamp' in df.columns else df['index'])
    df = df.set_index('timestamp')
    return df.sort_index()

In [6]:
total_times = []
per_sample_times = []

In [8]:
for path in dataset:
    df = load_df(path)
    
    model = NormalModel('','', mu_0=0, sigma_0=10, alpha_0=1, beta_0=1, gamma=1)
    start = datetime.now()

    start = datetime.now()
    for i, x in df['rtt'].items():
        model.log(i.timestamp()*1000, x)
    end = datetime.now()

    duration = end - start
    total_times.append(duration.total_seconds())
    per_sample_times.append(duration.total_seconds() / len(df))

    # file_path = pathlib.Path(path[18:])
    # file_dir = benchmark_dir / file_path.parent
    # if not file_dir.exists():
    #     file_dir.mkdir(parents=True)

    # out_path = file_dir / file_path.name
    # current = model.to_frame()['anomalies'].rename(model_name)
    # current.index = current.index.rename('timestamp')
    # if out_path.exists():
    #     existing = load_df(out_path)
    #     current = pd.concat([existing, current], axis=1)

    # current.to_csv(out_path)


In [9]:
names = [pathlib.Path(path).name for path in dataset]

In [10]:
stats_df = pd.DataFrame({f'time_{model_name}': total_times, f'per_sample_{model_name}': per_sample_times}, index=names)

In [11]:
stats_df.to_csv(benchmark_dir / f'stats_{model_name}.csv')